In [1]:
import numpy as np
import pandas as pd
np.set_printoptions(precision=6, suppress=True)

# Решение задачи Дирихле:
## Известно, что $u(x,y) = 2x^3y^3$
## Оно является решением задачи Дирихле:
### $\frac{\partial^2 u}{\partial x^2} + \frac{\partial^2 u}{\partial y^2} = -(\underbrace{-12xy(x^2+y^2)}_{f(x,y)})$
### $u(0,y) \equiv 0$
### $u(1,y) = 2y^3$
### $u(x,0) \equiv 0$
### $u(x,1) = 2x^3$

In [2]:
N = 11
a_x = 0
b_x = 1
a_y = 0
b_y = 1
h = (b_x-a_x)/(N - 1) # ширина сетки вместе с граничными узлами
iN = N - 2 # ширина сетки без граничных узлов

f = lambda x,y: -12*x*y*(x*x+y*y)
u1y = lambda y: 2*y**3
ux1 = lambda x: 2*x**3
u_expected = lambda x,y: 2*x**3*y**3

### Задаём начальное приближение с помощью начальных данных

In [3]:
def init_U(N):
    U = np.zeros(shape = (N, N))
    for i in range(N):
        U[i, N - 1] = ux1(i*h)
    for j in range(N):
        U[N - 1, j] = u1y(j*h)
    return U

def build_init_expected_and_F(N, h, u_expected, ux1, u1y, f):
    U = init_U(N)
    U_expected = U.copy() # задаём границы
    U_expected[1:N-1, 1:N-1] = np.array([[u_expected(i*h,j*h) for j in range(1, N-1)] for i in range(1, N-1)])
    F = np.array([[f(i*h, j*h) for j in range(1, N-1)] for i in range(1,N-1)])
    return {'init': U, 'expected': U_expected, 'F': F}

In [4]:
initials = build_init_expected_and_F(N, h, u_expected,ux1, u1y, f)
U_init = initials['init']
U_expected = initials['expected']
F = initials['F']

### Перечисляем параметры алгоритма в простейшем случае

In [5]:
tau = h*h/np.sin(np.pi * h) # оптимальный параметр

A = tau/(2*h*h)
B = tau/(h*h) + 1
C = tau/(2*h*h)

### Реализуем алгоритм метода переменных направлений

In [6]:
class AlternatingDirection:
    def __init__(self, A, B, C):
        self.A = A
        self.B = B
        self.C = C

    def __tridiag_alg_rowmode(self,U, F, N, h, tau, U_init):
        G = U_init.copy()
        U_new = U.copy()

        G[1:N-1, 1:N-1] = -U[1:N-1, 1:N-1] - (U[1:N-1, 2:N] - 2*U[1:N-1, 1:N-1] + U[1:N-1,0:N-2] + h*h*F)*tau/(2*h*h)

        s = np.zeros(shape = (N - 1, N))
        t = np.zeros(shape = (N - 1, N ))
        s[1:N-1, 0] = self.C/self.B
        t[1:N-1, 0] = -G[1:N-1, 0]/self.B
        for j in range(1, N-1):
            s[j, 1:N-1] = self.C/(self.B - self.A*s[j-1, 1:N-1])
            t[j, 1:N-1] = (self.A*t[j-1, 1:N-1] - G[j, 1:N-1])/(self.B - self.A*s[j-1, 1:N-1])
        for j in range(N-2, 0, -1):
            U_new[j, 1:N-1] = s[j, 1:N-1] * U_new[j+1, 1:N-1] + t[j, 1:N-1]
        return U_new

    def __tridiag_alg_colmode(self, U, F, N, h, tau, U_init):
        G = U_init.copy()
        U_new = U.copy()

        G[1:N-1, 1:N-1] = -U[1:N-1, 1:N-1] - (U[2:N, 1:N-1] - 2*U[1:N-1, 1:N-1] + U[0:N-2, 1:N-1] + h*h*F)*tau/(2*h*h)

        s = np.zeros(shape = (N, N - 1))
        t = np.zeros(shape = (N, N - 1))
        s[0, 1:N-1] = self.C/self.B
        t[0, 1:N-1] = -G[0, 1:N-1]/self.B

        s[1:N-1, 0] = self.C/self.B
        t[1:N-1, 0] = -G[1:N-1, 0]/self.B
        for j in range(1, N-1):
            s[1:N-1, j] = self.C/(self.B - self.A*s[1:N-1, j-1])
            t[1:N-1, j] = (self.A*t[1:N-1, j-1] - G[1:N-1, j])/(self.B - self.A*s[1:N-1, j-1])
        for j in range(N-2, 0, -1):
            U_new[1:N-1, j] = s[1:N-1, j] * U_new[1:N-1, j+1] + t[1:N-1, j]
        return U_new

    def __ADMethod_iter(self, U, F, N, h, tau, U_init):
        U_half = self.__tridiag_alg_rowmode(U, F, N, h, tau, U_init)
        return self.__tridiag_alg_colmode(U_half, F, N, h, tau, U_init)

    def __method_operator(self, U, N, h):
        return (U[0:N-2, 1:N-1] + U[2:N, 1:N-1] +
                U[1:N-1, 0:N-2] + U[1:N-1, 2:N] -
                                                4*U[1:N-1, 1:N-1])/(h*h)

    def solve(self, U_init, U_expected, F, eps = 1e-4, tau = None):
        N = U_init.shape[0]
        h = 1/(N - 1)

        if tau is None:
            tau = h*h/np.sin(np.pi * h)

        miniter = N*np.log(1/eps)/(2*np.pi)

        currU = U_init.copy()
        prevU = currU - 1
        iter = 1

        init_error = np.max(np.abs(U_init - U_expected))
        init_discrepancy = np.max(np.abs( self.__method_operator(U_init, N, h) + F ))
        rel_error = 1
        rel_discrepancy = 1

        output_list = list()

        while rel_error > eps and iter < miniter:
            prevU = currU.copy()
            currU = self.__ADMethod_iter(prevU, F, N, h, tau, U_init)

            curr_error = np.max(np.abs(currU - U_expected))
            curr_discrepancy = np.max(np.abs( self.__method_operator(currU, N, h) + F ))

            rel_error = curr_error/init_error
            rel_discrepancy = curr_discrepancy/init_discrepancy

            output_list+=[ [iter , curr_error, rel_error,
                            curr_discrepancy, rel_discrepancy,
                            np.max(np.abs(currU - prevU))] ]
            iter+=1
        output_list+=[ [' ', 'min_iters:' , miniter, ' ', ' ', ' '] ]
        return {'solution' : currU, 'output' : pd.DataFrame( data = output_list,
                                                             columns= ['k', 'Curr. discr.', 'Rel. discr.', 'Curr. error', 'Rel. error', '$ \lVert U_k - U_{k-1} \rVert $' ]).set_index(keys = 'k')}

In [7]:
solver = AlternatingDirection(A, B, C)
summary = solver.solve(U_init, U_expected, F)
solution = summary['solution']
summary['output']

,Curr. discr.,Rel. discr.,Curr. error,Rel. error,$ \lVert U_k - U_{k-1} \rVert $
k,,,,,
1,0.134477,0.126521,24.165676,0.087603,1.03642
2,0.046712,0.043949,15.822041,0.057357,0.116797
3,0.017071,0.016061,4.621396,0.016753,0.047364
4,0.009916,0.009329,2.149674,0.007793,0.01652
5,0.005033,0.004735,1.113859,0.004038,0.008585
6,0.002656,0.002499,0.382147,0.001385,0.004155
7,0.00147,0.001383,0.269616,0.000977,0.001834
8,0.000725,0.000682,0.08626,0.000313,0.001093
9,0.000405,0.000381,0.119963,0.000435,0.000443


In [8]:
solution

array([[0.      , 0.      , 0.      , 0.      , 0.      , 0.      ,
        0.      , 0.      , 0.      , 0.      , 0.      ],
       [0.      , 0.000002, 0.000018, 0.000057, 0.000131, 0.000253,
        0.000434, 0.000687, 0.001025, 0.001458, 0.002   ],
       [0.      , 0.000019, 0.000134, 0.000439, 0.001031, 0.002005,
        0.00346 , 0.00549 , 0.008194, 0.011664, 0.016   ],
       [0.      , 0.000063, 0.000445, 0.001472, 0.003468, 0.006759,
        0.011671, 0.018526, 0.027651, 0.039367, 0.054   ],
       [0.      , 0.00015 , 0.001049, 0.003479, 0.00821 , 0.016014,
        0.027658, 0.04391 , 0.06554 , 0.093313, 0.128   ],
       [0.      , 0.000293, 0.002041, 0.006784, 0.016026, 0.031269,
        0.054013, 0.085757, 0.128005, 0.182251, 0.25    ],
       [0.      , 0.000506, 0.00352 , 0.011713, 0.027683, 0.054023,
        0.093327, 0.148185, 0.22119 , 0.31493 , 0.432   ],
       [0.      , 0.000805, 0.00558 , 0.018584, 0.043944, 0.085775,
        0.148191, 0.235306, 0.351237, 0.500

In [9]:
U_expected

array([[0.      , 0.      , 0.      , 0.      , 0.      , 0.      ,
        0.      , 0.      , 0.      , 0.      , 0.      ],
       [0.      , 0.000002, 0.000016, 0.000054, 0.000128, 0.00025 ,
        0.000432, 0.000686, 0.001024, 0.001458, 0.002   ],
       [0.      , 0.000016, 0.000128, 0.000432, 0.001024, 0.002   ,
        0.003456, 0.005488, 0.008192, 0.011664, 0.016   ],
       [0.      , 0.000054, 0.000432, 0.001458, 0.003456, 0.00675 ,
        0.011664, 0.018522, 0.027648, 0.039366, 0.054   ],
       [0.      , 0.000128, 0.001024, 0.003456, 0.008192, 0.016   ,
        0.027648, 0.043904, 0.065536, 0.093312, 0.128   ],
       [0.      , 0.00025 , 0.002   , 0.00675 , 0.016   , 0.03125 ,
        0.054   , 0.08575 , 0.128   , 0.18225 , 0.25    ],
       [0.      , 0.000432, 0.003456, 0.011664, 0.027648, 0.054   ,
        0.093312, 0.148176, 0.221184, 0.314928, 0.432   ],
       [0.      , 0.000686, 0.005488, 0.018522, 0.043904, 0.08575 ,
        0.148176, 0.235298, 0.351232, 0.500

### Убедимся, что параметр по умолчанию является оптимальным

In [10]:
solver.solve(U_init, U_expected, F, tau = tau - 0.01) ['output']

,Curr. discr.,Rel. discr.,Curr. error,Rel. error,$ \lVert U_k - U_{k-1} \rVert $
k,,,,,
1,0.126284,0.118812,18.416615,0.066762,1.083583
2,0.028212,0.026543,3.31318,0.012011,0.117941
3,0.026972,0.025377,3.745297,0.013577,0.032124
4,0.032491,0.030569,3.285629,0.011911,0.013101
5,0.036199,0.034058,3.461902,0.01255,0.007896
6,0.038775,0.036481,3.419232,0.012395,0.00468
7,0.040078,0.037707,3.442614,0.01248,0.002762
8,0.040889,0.038470,3.44046,0.012472,0.001665
9,0.041353,0.038906,3.444024,0.012485,0.000992


In [11]:
solver.solve(U_init, U_expected, F, tau = tau + 0.01) ['output']

,Curr. discr.,Rel. discr.,Curr. error,Rel. error,$ \lVert U_k - U_{k-1} \rVert $
k,,,,,
1,0.142221,0.133807,64.716393,0.234604,0.926517
2,0.152181,0.143178,56.366109,0.204333,0.166717
3,0.060646,0.057058,27.595703,0.100037,0.091535
4,0.065555,0.061677,25.300767,0.091718,0.040341
5,0.050425,0.047441,15.783653,0.057217,0.026552
6,0.039984,0.037618,14.551516,0.052751,0.013273
7,0.043657,0.041074,10.582089,0.038361,0.009968
8,0.037272,0.035067,9.691213,0.035132,0.006385
9,0.039509,0.037172,8.04339,0.029158,0.004308


#### Видно, что если отойти от параметра по умолчанию, то все показатели ухудшаются, то есть понадобиться выполнить больше итераций, чтобы достичь такого же результата, как в случае с оптимальным параметром
